In [12]:

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from matplotlib.patches import Patch
from sklearn.decomposition import PCA
from tensorflow import keras
import pandas as pd
import random

In [13]:
def set_seed(seed=42):
  random.seed(seed)
  np.random.seed(seed)
  tf.experimental.numpy.random.seed(seed)

set_seed()
project_dir = os.path.dirname(os.path.dirname(os.getcwd()))
project_dir = os.path.join(project_dir, 'plant-diseases-identification')

In [14]:
model_mn2 = keras.applications.MobileNetV2(
    input_shape=None, alpha=1.0, include_top=True, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000
)
layer_name = 'block_15_add'
feature_extractor = keras.models.Model(inputs=model_mn2.input, outputs=model_mn2.get_layer(layer_name).output)

In [16]:
models_path = os.path.join(project_dir, 'models')
model_mobilenetv2_regularized = tf.keras.models.load_model(os.path.join(models_path, 'model_mobilenetv2_regularized_pvd'))
model_squeeze_excite_resnet = tf.keras.models.load_model(os.path.join(models_path, 'model_squeeze_excite_resnet_plantvillage'))
model_pca_based = tf.keras.models.load_model(os.path.join(models_path, 'model_pca_based_plantvillage'))
model_pca_based_sepconv = tf.keras.models.load_model(os.path.join(models_path, 'model_pca_based_sepconv_plantvillage'))
model_lda_based = tf.keras.models.load_model(os.path.join(models_path, 'model_lda_based_plantvillage'))
model_lda_based_sepconv = tf.keras.models.load_model(os.path.join(models_path, 'model_lda_based_sepconv_plantvillage'))


In [142]:
diseases_names = []
dataset_path = os.path.join(project_dir, 'datasets', 'encoded', 'PlantDoc')

for diseases in os.listdir(os.path.join(dataset_path, 'train')):
    diseases_names += [diseases.split('.npy')[0]]

diseases_names.sort()
plant_name_to_id = {plant: i for i, plant in enumerate(diseases_names)}

In [143]:
instances = {}
labels = {}

for set_kind in ["test", "train", "val"]:
    X_per_class = []
    y_per_class = []

    for i, file in enumerate(os.listdir(os.path.join(dataset_path, set_kind))):
        print(f"\r[{i}] {set_kind}/{file}", end='')
        plants_encoded = np.load(os.path.join(dataset_path, set_kind, file))
        plant_name = file.split('.npy')[0]

        X_per_class += [plants_encoded.reshape([plants_encoded.shape[0], 49, 160])]
        y_per_class += [plant_name_to_id[plant_name] for _ in range(plants_encoded.shape[0])]

    instances[set_kind] = np.concatenate(X_per_class)
    labels[set_kind] = np.array(y_per_class)

[26] val/Tomato Septoria leaf spot.npyyyy

In [144]:
n_classes = len(diseases_names)

X_train = instances['train']
y_train = np.eye(n_classes)[labels['train']]


X_valid = instances['val']
y_valid = np.eye(n_classes)[labels['val']]

X_test = instances['test']
y_test = np.eye(n_classes)[labels['test']]

In [145]:
print(X_train[0].shape)
print(X_train[..., 0].shape)

(49, 160)
(28983, 49)


In [146]:
from sklearn.decomposition import PCA
print(X_train[..., 0].shape)


(28983, 49)


In [147]:
pca_per_filter = [
    PCA(n_components=5).fit(X_train[..., i])
    for i in range(X_train.shape[-1])
]

In [148]:
all_means = np.array([filter.mean_ for filter in pca_per_filter])
all_means = all_means.T.reshape([1, 49, 160])

all_components = np.array([filter.components_ for filter in pca_per_filter])
all_components = np.transpose(all_components, [2, 1, 0])

filterwise_dot = np.einsum('ij...,jk...->...ik', X_train - all_means, all_components)
filterwise_dot = np.transpose(filterwise_dot, axes=[1, 2, 0])

X_train_pca = filterwise_dot

all_means = np.array([filter.mean_ for filter in pca_per_filter])
all_means = all_means.T.reshape([1, 49, 160])

all_components = np.array([filter.components_ for filter in pca_per_filter])
all_components = np.transpose(all_components, [2, 1, 0])

filterwise_dot = np.einsum('ij...,jk...->...ik', X_valid - all_means, all_components)
# equivalent to einsum tf.stack([A[..., i] @ B[..., i] for i in tf.range(160)], axis=-1)
filterwise_dot = np.transpose(filterwise_dot, axes=[1, 2, 0])

X_valid_pca = filterwise_dot

In [177]:
'''
models_path = os.path.join(project_dir, 'models')
model_mobilenetv2_regularized = tf.keras.models.load_model(os.path.join(models_path, 'model_mobilenetv2_regularized_pvd'))
model_squeeze_excite_resnet = tf.keras.models.load_model(os.path.join(models_path, 'model_squeeze_excite_resnet_plantvillage'))
model_pca_based = tf.keras.models.load_model(os.path.join(models_path, 'model_pca_based_plantvillage'))
model_pca_based_sepconv = tf.keras.models.load_model(os.path.join(models_path, 'model_pca_based_sepconv_plantvillage'))
model_lda_based = tf.keras.models.load_model(os.path.join(models_path, 'model_lda_based_plantvillage'))
model_lda_based_sepconv = tf.keras.models.load_model(os.path.join(models_path, 'model_lda_based_sepconv_plantvillage'))
'''

for layer in model_lda_based_sepconv.layers:
    layer.trainable = False

# Add a new output layer with the desired number of neurons
output_layer = tf.keras.layers.Dense(units=27, activation="softmax", name="predictions")(model_lda_based_sepconv.layers[-2].output)

# Combine the pre-trained model with the new output layer
model = tf.keras.models.Model(inputs=model_lda_based_sepconv.input, outputs=output_layer)

# Compile the model with a suitable optimizer and loss function
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

In [178]:
model.summary()


Model: "model_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 5, 160)]          0         
                                                                 
 batch_normalization_2 (Batc  (None, 5, 160)           640       
 hNormalization)                                                 
                                                                 
 separable_conv1d (Separable  (None, 1, 320)           156320    
 Conv1D)                                                         
                                                                 
 elu_2 (ELU)                 (None, 1, 320)            0         
                                                                 
 flatten (Flatten)           (None, 320)               0         
                                                                 
 dropout_2 (Dropout)         (None, 320)               0  

In [113]:
set_seed()
filepath = os.path.join(project_dir, 'models', 'model_mobilenetv2_regularized_plantdoc_pretrained_pvd')
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                monitor='val_loss',
                                                verbose=1,
                                                save_best_only=True,
                                                mode='min')

lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.66, patience=1)
idx = np.arange(X_train.shape[0])
np.random.shuffle(idx)

X_train_shuffled = X_train[idx]
y_train_shuffled = y_train[idx]

history = model.fit(
    X_train_shuffled, y_train_shuffled,
    epochs=10,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler, checkpoint]
)

Epoch 1/10
906/906 [==============================] - ETA: 0s - loss: 2.5077 - accuracy: 0.3443  

INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_mobilenetv2_regularized_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_mobilenetv2_regularized_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_mobilenetv2_regularized_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_mobilenetv2_regularized_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_mobilenetv2_regularized_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_mobilenetv2_regularized_plantdoc_pretrained_pvd\assets


In [117]:
set_seed()
filepath = os.path.join(project_dir, 'models', 'model_squeeze_excite_resnet_plantdoc_pretrained_pvd')
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                monitor='val_loss',
                                                verbose=1,
                                                save_best_only=True,
                                                mode='min')

idx = np.arange(X_train.shape[0])
np.random.shuffle(idx)

X_train_shuffled = X_train[idx]
y_train_shuffled = y_train[idx]

model.fit(
    X_train_shuffled, y_train_shuffled,
    epochs=10,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler, checkpoint]
)

Epoch 1/10
899/906 [============================>.] - ETA: 0s - loss: 2.4012 - accuracy: 0.3297   

INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_squeeze_excite_resnet_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_squeeze_excite_resnet_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_squeeze_excite_resnet_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_squeeze_excite_resnet_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_squeeze_excite_resnet_plantdoc_pretrained_pvd\assets


In [126]:
set_seed()
filepath = os.path.join(project_dir, 'models', 'model_pca_based_plantdoc_pretrained_pvd')
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                monitor='val_loss',
                                                verbose=1,
                                                save_best_only=True,
                                                mode='min')

lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)

model.fit(
    X_train_pca_shuffled, y_train_shuffled,
    epochs=10,
    validation_data=(X_valid_pca, y_valid),
    callbacks=[lr_scheduler, checkpoint]
)


Epoch 1/10
903/906 [============================>.] - ETA: 0s - loss: 2.6369 - accuracy: 0.2512   

INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_pca_based_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_pca_based_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_pca_based_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_pca_based_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_pca_based_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_pca_based_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identificati

In [152]:
set_seed()

idx = np.arange(X_train_pca.shape[0])
np.random.shuffle(idx)

X_train_pca_shuffled = X_train_pca[idx]
y_train_shuffled = y_train[idx]

filepath = os.path.join(project_dir, 'models', 'model_pca_based_sepconv_plantdoc_pretrained_pvd')
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                monitor='val_loss',
                                                verbose=1,
                                                save_best_only=True,
                                                mode='min')

lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)

model.fit(
    X_train_pca_shuffled, y_train_shuffled,
    epochs=10,
    validation_data=(X_valid_pca, y_valid),
    callbacks=[lr_scheduler, checkpoint]
)

Epoch 1/10
906/906 [==============================] - ETA: 0s - loss: 1.6753 - accuracy: 0.4776  

INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_pca_based_sepconv_plantdoc_pretrained_pvd\assets


In [164]:
diseases_names = []
dataset_path = os.path.join(project_dir, 'datasets', 'encoded', 'PlantDoc')

for diseases in os.listdir(os.path.join(dataset_path, 'train')):
    diseases_names += [diseases.split('.npy')[0]]

diseases_names.sort()
plant_name_to_id = {plant: i for i, plant in enumerate(diseases_names)}

In [165]:
instances = {}
labels = {}

for set_kind in ["test", "train", "val"]:
    X_per_class = []
    y_per_class = []

    for i, file in enumerate(os.listdir(os.path.join(dataset_path, set_kind))):
        print(f"\r[{i}] {set_kind}/{file}", end='')
        plants_encoded = np.load(os.path.join(dataset_path, set_kind, file))
        plant_name = file.split('.npy')[0]

        X_per_class += [plants_encoded.reshape([plants_encoded.shape[0], 49, 160])]
        y_per_class += [plant_name_to_id[plant_name] for _ in range(plants_encoded.shape[0])]

    instances[set_kind] = np.concatenate(X_per_class)
    labels[set_kind] = np.array(y_per_class)

[26] val/Tomato Septoria leaf spot.npyyyy

In [166]:
n_classes = len(diseases_names)

X_train = instances['train']
y_train = np.eye(n_classes)[labels['train']]


X_valid = instances['val']
y_valid = np.eye(n_classes)[labels['val']]

X_test = instances['test']
y_test = np.eye(n_classes)[labels['test']]

In [167]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [168]:
lda_per_filter = [
    LDA(n_components=5).fit(X_train[..., i], np.argmax(y_train, axis=1))
    for i in range(X_train.shape[-1])
]

In [169]:
all_means = np.array([filter.xbar_ for filter in lda_per_filter])
all_means = all_means.T.reshape([1, 49, 160])

all_components = np.array([filter.scalings_[:, :5] for filter in lda_per_filter])
all_components = np.transpose(all_components, [1, 2, 0])

In [170]:
filterwise_dot = np.einsum('ij...,jk...->...ik', X_train - all_means, all_components)
filterwise_dot = np.transpose(filterwise_dot, axes=[1, 2, 0])

X_train_lda = filterwise_dot

filterwise_dot = np.einsum('ij...,jk...->...ik', X_valid - all_means, all_components)
filterwise_dot = np.transpose(filterwise_dot, axes=[1, 2, 0])

X_valid_lda = filterwise_dot

filterwise_dot = np.einsum('ij...,jk...->...ik', X_test - all_means, all_components)
filterwise_dot = np.transpose(filterwise_dot, axes=[1, 2, 0])

X_test_lda = filterwise_dot

In [171]:
idx = np.arange(X_train_lda.shape[0])
np.random.shuffle(idx)

X_train_lda_shuffled = X_train_lda[idx]
y_train_shuffled = y_train[idx]

X_train_lda_shuffled_flat = X_train_lda_shuffled.reshape(-1, 5 * 160)
X_valid_lda_flat = X_valid_lda.reshape(-1, 5 * 160)
X_test_lda_flat = X_test_lda.reshape(-1, 5 * 160)

In [176]:
set_seed()
filepath = os.path.join(project_dir, 'models', 'model_lda_based_plantdoc_pretrained_pvd')
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                monitor='val_loss',
                                                verbose=1,
                                                save_best_only=True,
                                                mode='min')

lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)

model.fit(
    X_train_lda_shuffled_flat, y_train_shuffled,
    epochs=10,
    validation_data=(X_valid_lda_flat, y_valid),
    callbacks=[lr_scheduler, checkpoint]
)


Epoch 1/10
897/906 [============================>.] - ETA: 0s - loss: 2.7234 - accuracy: 0.2171   

INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_lda_based_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_lda_based_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_lda_based_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_lda_based_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_lda_based_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_lda_based_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identificati

In [179]:
set_seed()
filepath = os.path.join(project_dir, 'models', 'model_lda_based_sepconv_plantdoc_pretrained_pvd')
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                monitor='val_loss',
                                                verbose=1,
                                                save_best_only=True,
                                                mode='min')

lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)

model.fit(
    X_train_lda_shuffled, y_train_shuffled,
    epochs=10,
    validation_data=(X_valid_lda, y_valid),
    callbacks=[lr_scheduler, checkpoint]
)

Epoch 1/10
901/906 [============================>.] - ETA: 0s - loss: 2.5824 - accuracy: 0.2646   

INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_lda_based_sepconv_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_lda_based_sepconv_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_lda_based_sepconv_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_lda_based_sepconv_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_lda_based_sepconv_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\ASROCK\PycharmProjects\plant-diseases-identification\models\model_lda_based_sepconv_plantdoc_pretrained_pvd\assets
INFO:tensorflow:Assets written to: C:\Users\AS